In [19]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import os
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch
set_gpu()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Num GPUs Available:  1


In [20]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,feat,feat_sq = pickle.load(f)

In [26]:
## Training 
sub = 2
train_grp = 2
n_train = 'fullallmix4'
train_scale = 5
cv_type = 'manual'
scaler_load = False
feat_type = 'mav'

ind = (params[:,0] == sub) & (params[:,3] == train_grp)

x_train, x_test, x_valid, p_train, p_test, p_valid = prd.train_data_split(raw,params,sub,sub_type,dt=cv_type,load=True,train_grp=train_grp)

emg_scale = np.ones((np.size(x_train,1),1))
for i in range(np.size(x_train,1)):
    emg_scale[i] = 5/np.max(np.abs(x_train[:,i,:]))
x_train = x_train*emg_scale
x_valid = x_valid*emg_scale

x_train_noise, x_train_clean, y_train_clean = prd.add_noise(x_train, p_train, sub, n_train, train_scale)
x_valid_noise, x_valid_clean, y_valid_clean = prd.add_noise(x_train, p_train, sub, n_train, train_scale)

# shuffle data to make even batches
x_train_noise, x_train_clean, y_train_clean = shuffle(x_train_noise, x_train_clean, y_train_clean, random_state = 0)

# Extract features
scaler = MinMaxScaler(feature_range=(0,1))
x_train_noise_cnn, scaler = prd.extract_scale(x_train_noise,scaler,scaler_load,ft=feat_type,emg_scale=emg_scale) 
x_train_clean_cnn, _ = prd.extract_scale(x_train_clean,scaler,ft=feat_type,emg_scale=emg_scale)
x_valid_noise_cnn, _ = prd.extract_scale(x_valid_noise,scaler,ft=feat_type,emg_scale=emg_scale)
x_valid_clean_cnn, _ = prd.extract_scale(x_valid_clean,scaler,ft=feat_type,emg_scale=emg_scale)
x_train_noise_cnn = x_train_noise_cnn.astype('float32')
x_valid_noise_cnn = x_valid_noise_cnn.astype('float32')

# reshape data for nonconvolutional network
x_train_noise_mlp = x_train_noise_cnn.reshape(x_train_noise_cnn.shape[0],-1)
x_valid_noise_mlp = x_valid_noise_cnn.reshape(x_valid_noise_cnn.shape[0],-1)


# create batches
trainmlp_ds = tf.data.Dataset.from_tensor_slices((x_train_noise_mlp, y_train_clean)).batch(128)
testmlp_ds = tf.data.Dataset.from_tensor_slices((x_valid_noise_mlp, y_valid_clean)).batch(128)
traincnn_ds = tf.data.Dataset.from_tensor_slices((x_train_noise_cnn, y_train_clean)).batch(128)
testcnn_ds = tf.data.Dataset.from_tensor_slices((x_valid_noise_cnn, y_valid_clean)).batch(128)

Loading training data: traindata_manual/AB2_traindata_2.p


In [27]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

## TRAIN TEST MLP
@tf.function
def train_mlp(x, y):
    with tf.GradientTape() as tape:
        y_out = mlp(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, mlp.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_mlp(x, y):
    y_out = mlp(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

## TRAIN TEST MLP BETA
@tf.function
def train_mlpbeta(x, y):
    with tf.GradientTape() as tape:
        y_out = mlp_beta(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, mlp_beta.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlp_beta.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_mlpbeta(x, y):
    y_out = mlp_beta(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

## TRAIN TEST CNN
@tf.function
def train_cnn(x, y):
    with tf.GradientTape() as tape:
        y_out = cnn(x)
        loss = loss_fn(y, y_out)
    gradients = tape.gradient(loss, cnn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, cnn.trainable_variables))

    train_loss(loss)
    train_accuracy(y, y_out)

@tf.function
def test_cnn(x, y):
    y_out = cnn(x)
    t_loss = loss_fn(y, y_out)

    test_loss(t_loss)
    test_accuracy(y, y_out)

In [28]:
EPOCHS = 30
models = ['mlp','mlpbeta','cnn']

for model in models:
    print('Training ' + model)
    for epoch in range(EPOCHS):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        # Train MLP
        if model == 'mlp':
            for x, y in trainmlp_ds:
                train_mlp(x, y)
            for x_test, y_test in testmlp_ds:
                test_mlp(x_test, y_test)
        # Train MLP Beta
        elif model == 'mlpbeta':
            for x, y in trainmlp_ds:
                train_mlpbeta(x, y)
            for x_test, y_test in testmlp_ds:
                test_mlpbeta(x_test, y_test)
        # Train CNN
        elif model == 'cnn':
            for x, y in traincnn_ds:
                train_cnn(x,y)
            for x_test, y_test in testcnn_ds:
                test_cnn(x_test, y_test)

        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

Training mlp
Epoch 1, Loss: 1.937026023864746, Accuracy: 20.428571701049805, Test Loss: 1.9195363521575928, Test Accuracy: 24.103174209594727
Epoch 2, Loss: 1.8813036680221558, Accuracy: 29.0079345703125, Test Loss: 1.8396662473678589, Test Accuracy: 32.666664123535156
Epoch 3, Loss: 1.8149486780166626, Accuracy: 35.42857360839844, Test Loss: 1.7933012247085571, Test Accuracy: 37.515872955322266
Epoch 4, Loss: 1.7852373123168945, Accuracy: 37.5079345703125, Test Loss: 1.7749580144882202, Test Accuracy: 38.31349182128906
Epoch 5, Loss: 1.7723084688186646, Accuracy: 38.55952453613281, Test Loss: 1.7653300762176514, Test Accuracy: 39.32936477661133
Epoch 6, Loss: 1.7646064758300781, Accuracy: 39.408729553222656, Test Loss: 1.7590690851211548, Test Accuracy: 39.988094329833984
Epoch 7, Loss: 1.7587984800338745, Accuracy: 40.19047546386719, Test Loss: 1.7538120746612549, Test Accuracy: 40.72222137451172
Epoch 8, Loss: 1.7515652179718018, Accuracy: 41.18253707885742, Test Loss: 1.74537253379

In [29]:
mlp_enc = mlp.get_layer(name='enc')
mlpbeta_enc = mlp_beta.get_layer(name='enc')
cnn_enc = cnn.get_layer(name='enc')

mlp_aligned = mlp_enc(x_train_noise_mlp).numpy()
mlp_beta_aligned = mlpbeta_enc(x_train_noise_mlp).numpy()
cnn_aligned = cnn_enc(x_train_noise_cnn).numpy()

y_train_aligned = np.argmax(y_train_clean, axis=1)[...,np.newaxis]
w_mlp, c_mlp,_, _, _ = train_lda(mlp_aligned,y_train_aligned)
w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlp_beta_aligned,y_train_aligned)
w_cnn, c_cnn,_, _, _ = train_lda(cnn_aligned,y_train_aligned)

In [30]:
test_grp = 4
cv_type = 'manual'
n_test = 'partposrealmixeven24'

with open('real_noise/all_real_noise.p', 'rb') as f:
    real_noise_temp, _ = pickle.load(f)

_, x_test, _, _, p_test, _ = prd.train_data_split(raw,params,sub,sub_type,dt=cv_type,train_grp=test_grp)
clean_size = int(np.size(x_test,axis=0))
x_test = x_test*emg_scale

x_test_noise, x_test_clean, y_test_clean = prd.add_noise(x_test, p_test, sub, n_test, 1, real_noise=real_noise_temp, emg_scale = emg_scale)
x_test_cnn, _ = prd.extract_scale(x_test_noise,scaler,ft=feat_type,emg_scale=emg_scale)
x_test_cnn = x_test_cnn.astype('float32')
x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)

mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
y_test_aligned = np.argmax(y_test_clean, axis=1)[...,np.newaxis]

test_accuracy.reset_states()
test_mlp(x_test_mlp[:clean_size,...],y_test_clean[:clean_size,...])
acc = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
print(
    f'Clean: '
    f'MLP Accuracy: {test_accuracy.result() * 100},'
    f'MLP-LDA Accuracy: {acc * 100}'
)

test_accuracy.reset_states()
test_mlp(x_test_mlp[clean_size:,...],y_test_clean[clean_size:,...])
acc = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'Noisy: '
    f'MLP Accuracy: {test_accuracy.result() * 100},'
    f'MLP-LDA Accuracy: {acc * 100}'
)

test_accuracy.reset_states()
test_mlpbeta(x_test_mlp[:clean_size,...],y_test_clean[:clean_size,...])
acc = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
print(
    f'Clean: '
    f'MLP-Beta Accuracy: {test_accuracy.result() * 100},'
    f'MLP-Beta-LDA Accuracy: {acc * 100}'
)

test_accuracy.reset_states()
test_mlpbeta(x_test_mlp[clean_size:,...],y_test_clean[clean_size:,...])
acc = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'Noisy: '
    f'MLP-Beta Accuracy: {test_accuracy.result() * 100},'
    f'MLP-Beta-LDA Accuracy: {acc * 100}'
)

test_accuracy.reset_states()
test_cnn(x_test_cnn[:clean_size,...],y_test_clean[:clean_size,...])
acc = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
print(
    f'Clean: '
    f'CNN Accuracy: {test_accuracy.result() * 100},'
    f'CNN-LDA Accuracy: {acc * 100}'
)

test_accuracy.reset_states()
test_cnn(x_test_cnn[clean_size:,...],y_test_clean[clean_size:,...])
acc = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'Noisy: '
    f'CNN Accuracy: {test_accuracy.result() * 100},'
    f'CNN-LDA Accuracy: {acc * 100}'
)


Loading training data: traindata_manual/AB2_traindata_4.p

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autoc